In [ ]:
!!git clone -b OpenKE-PyTorch https://github.com/thunlp/OpenKE
%cd OpenKE/
!mkdir checkpoint
!mkdir result
%cd openke
!bash ./make.sh

In [ ]:
%cd /content/OpenKE/benchmarks/
!mkdir RezoJDM16K
!wget -q https://github.com/ContentSide/French_Knowledge_Graph/raw/master/benchmarks/RezoJDM16K/1-1.txt -O RezoJDM16K/1-1.txt
!wget -q https://github.com/ContentSide/French_Knowledge_Graph/raw/master/benchmarks/RezoJDM16K/1-n.txt -O RezoJDM16K/1-n.txt 
!wget -q https://github.com/ContentSide/French_Knowledge_Graph/raw/master/benchmarks/RezoJDM16K/entity2id.txt -O RezoJDM16K/entity2id.txt
!wget -q https://github.com/ContentSide/French_Knowledge_Graph/raw/master/benchmarks/RezoJDM16K/n-1.txt -O RezoJDM16K/n-1.txt
!wget -q https://github.com/ContentSide/French_Knowledge_Graph/raw/master/benchmarks/RezoJDM16K/n-n.txt -O RezoJDM16K/n-n.txt
!wget -q https://github.com/ContentSide/French_Knowledge_Graph/raw/master/benchmarks/RezoJDM16K/relation2id.txt -O RezoJDM16K/relation2id.txt
!wget -q https://github.com/ContentSide/French_Knowledge_Graph/raw/master/benchmarks/RezoJDM16K/test2id.txt -O RezoJDM16K/test2id.txt
!wget -q https://github.com/ContentSide/French_Knowledge_Graph/raw/master/benchmarks/RezoJDM16K/test2id_all.txt -O RezoJDM16K/test2id_all.txt
!wget -q https://github.com/ContentSide/French_Knowledge_Graph/raw/master/benchmarks/RezoJDM16K/train2id.txt -O RezoJDM16K/train2id.txt
!wget -q https://github.com/ContentSide/French_Knowledge_Graph/raw/master/benchmarks/RezoJDM16K/type_constrain.txt -O RezoJDM16K/type_constrain.txt
!wget -q https://github.com/ContentSide/French_Knowledge_Graph/raw/master/benchmarks/RezoJDM16K/valid2id.txt -O RezoJDM16K/valid2id.txt
%cd ..

In [ ]:
import numpy as np
import openke
from openke.config import Trainer, Tester
from openke.module.model import TransD
from openke.module.loss import MarginLoss
from openke.module.strategy import NegativeSampling
from openke.data import TrainDataLoader, TestDataLoader

# dataloader for training
train_dataloader = TrainDataLoader(
	in_path="./benchmarks/RezoJDM16K/", 
	nbatches=100,
	threads=8, 
	sampling_mode="normal", 
	bern_flag=1, 
	filter_flag=1, 
	neg_ent=25,
	neg_rel=0
)

# dataloader for test
test_dataloader = TestDataLoader("./benchmarks/RezoJDM16K/", "link")

# define the model
transd = TransD(
	ent_tot=train_dataloader.get_ent_tot(),
	rel_tot=train_dataloader.get_rel_tot(),
	dim_e=200, 
	dim_r=200, 
	p_norm=1, 
	norm_flag=True
)


# define the loss function
model = NegativeSampling(
	model = transd, 
	loss = MarginLoss(margin = 4.0),
	batch_size = train_dataloader.get_batch_size()
)

# train the model
trainer = Trainer(
	model=model,
	data_loader=train_dataloader,
	train_times=50,
	alpha=1.0,
	use_gpu=True
)
trainer.run()

# save everything

# model
transd.save_checkpoint('./checkpoint/transd.ckpt')

# embeddings as np arrays
ent_embeddings = transd.ent_embeddings.weight.detach().cpu().numpy()
rel_embeddings = transd.rel_embeddings.weight.detach().cpu().numpy()
np.save("./benchmarks/RezoJDM16K/entity2vec200.npy", ent_embeddings)
np.save("./benchmarks/RezoJDM16K/relation2vec200.npy", rel_embeddings)

# embeddings as txt
relation_embs = np.load("./benchmarks/RezoJDM16K/relation2vec200.npy")
with open("./benchmarks/RezoJDM16K/relation2vec200.init", "w") as fp:
    for relation_emb in list(relation_embs):
        line = "\t".join(str(w) for w in list(relation_emb))
        fp.write(f"{line}\t\n")
entity_embs = np.load("./benchmarks/RezoJDM16K/entity2vec200.npy")
with open("./benchmarks/RezoJDM16K/entity2vec200.init", "w") as fp:
    for entity_emb in list(entity_embs):
        line = "\t".join(str(w) for w in list(entity_emb))
        fp.write(f"{line}\t\n")

# test the model
transd.load_checkpoint('./checkpoint/transd.ckpt')
tester = Tester(
	  model=transd,
	  data_loader=test_dataloader,
	  use_gpu=True
)
tester.run_link_prediction(type_constrain=False)

In [ ]:
%cd benchmarks/RezoJDM16K/
!zip embeddings-RezoJDM16K-transd.zip relation2vec200.init entity2vec200.init

In [ ]:
from google.colab import files
files.download("embeddings-RezoJDM16K-transd.zip")